In [19]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [20]:
import mtt
from os.path import join, realpath, split
from os import getcwd
sbfile = join(realpath(getcwd()),'inhibited-rev.sb')
wiring = mtt.Wiring.fromFile(sbfile)
wiring

In [21]:
model = mtt.MTT(wiring)
model.draw(2)

# Digitized current config

In [22]:
for block in wiring.blocks:
    print(block.getDigitizedParameterString(model))

# Simulink Export

In [23]:
model.exportToSimulink(join(realpath(getcwd()),'simulink','bottsmorales_inh_rev'), 'bottsmorales_inh_rev', 2)

# Block Simulation

In [24]:
sbml = model.toSBML()
# print(sbml)
from roadrunner import RoadRunner
with open('out-sbml-botts-morales-inh-rev.xml', 'w') as f:
    f.write(sbml)
xmod = RoadRunner(sbml)
xmod.reset()
xmod.simulate(0,10,1000,selections=['time']+xmod.getFloatingSpeciesIds())
xmod.plot()

In [25]:
# # block simulation
# model.reset()
# for k in range(3):
#     print('block {}: rate_fw = {}'.format(k,model.blocks[k].rate_fw))
#     print('block {}: rate = {}'.format(k,model.blocks[k].rate))
# wiring.blocks[1].Btot.setValue(50.)
# mtt_results = model.simulate(0,10,100)
# model.plot(xtitle='time(s)', ytitle='conc.(nA)')

# SBML Simulation

In [26]:
# SBML comparison
import tellurium as te
with open(sbfile) as f:
    rr_model = te.loada(f.read())
rr_results = rr_model.simulate(0,10,1000)
rr_model.plot(xtitle='time(s)', ytitle='conc.(nA)')

In [27]:
nrows = 2
ncols = 1
fig,axes = plt.subplots(nrows, ncols, sharex='col', sharey='row', figsize=(6,8))

for z,sb in enumerate(['i0.sb','i1.sb','i2.sb','i3.sb','i4.sb']):
    sb = join(realpath(getcwd()),sb)
    wiring = mtt.Wiring.fromFile(sb)
    model = mtt.MTT(wiring)
    sbml = model.toSBML()
    xmod = RoadRunner(sbml)
    mtt_results = xmod.simulate(0,10,1000,selections=['time','P'])
    
    with open(sb) as f:
        rr_model = te.loada(f.read())
    rr_results = rr_model.simulate(0,10,1000,selections=['time','P'])
    
    col = 0
    for row,(r,title) in enumerate(zip((mtt_results,rr_results),('Block Simulation','SBML Simulation'))):
        a = axes[row]
        a.set_title(title, fontsize=14)
        if row == nrows-1:
            a.set_xlabel('time(s)', fontsize=14)
        a.set_ylabel('Product conc.(nA)', fontsize=14)
        for k in range(1,r.shape[1]):
            a.plot(r[:,0], r[:,k], label='I = {:.0f}'.format(rr_model.I), color='C'+str(z), linewidth=2)
        handles, labels = a.get_legend_handles_labels()
fig.legend(handles, labels, loc='lower right')

pdfname = split(sbfile)[1].split('.')[0]+'.pdf'
plt.savefig(pdfname, format='pdf')